In [1]:
import rasterio
from glob import glob
import numpy as np
import pandas as pd
import pickle
import os
import multiprocessing as mp

In [2]:
def calc_mean_std(kwargs):
    i_band = kwargs["i_band"]
    band = kwargs["band"]
    clip_val = 1.5
    
    chips = glob("/data/chips_filtered/*_merged.tif")
    means = []
    stds = []
    
    print("Bands:", band)
    vals_all = np.ndarray([0])
    for k in range(len(chips)):
        file = chips[k]
        with rasterio.open(file) as src:
            vals = src.read(band).flatten()
            vals_all = np.concatenate([vals_all, vals])
    band_mean = vals_all.mean()
    band_std = vals_all.std()
    vals_all_clipped = np.clip(vals_all, np.nanpercentile(vals_all, clip_val), 
                               np.nanpercentile(vals_all, 100 - clip_val))
    band_min = np.min(vals_all_clipped)
    band_max = np.max(vals_all_clipped)
    f= open("/data/band_stats/band_values_" + str(i_band) + ".txt","w+")
    f.write("Mean: %f\r\n" % (band_mean))
    f.write("Std: %f\r\n" % (band_std))
    f.write("Min: %f\r\n" % (band_min))
    f.write("Max: %f\r\n" % (band_max))
    f.close
    
    with open("/data/band_stats/band_values_" + str(i_band) + '.list', 'wb') as file:
        pickle.dump(vals_all, file)

In [3]:
i_band = list(range(1, 7))
band_ids = [[1,7,13], [2,8,14], [3,9,15], [4,10,16], [5,11,17], [6,12,18]]
df = pd.DataFrame({"i_band": i_band, "band": band_ids})

In [4]:
with mp.Pool(processes=6) as pool:
    pool.map(calc_mean_std, df.to_dict('records'))

Bands:Bands:Bands:Bands:Bands:Bands:      [1, 7, 13][3, 9, 15][2, 8, 14][4, 10, 16][5, 11, 17][6, 12, 18]





